# FileSystem (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
// #r "nuget:FSharp.Control.AsyncSeq,3.2.1"
// #r "nuget:System.Reactive,5.0.0"
// #r "nuget:System.Reactive.Linq,6.0.0-preview.1"

#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/5.0.0/lib/net5.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/Async.fs
#!import ../nbs/AsyncSeq.fs
#!import ../nbs/Runtime.fs

In [ ]:
open Common

## Operators

In [ ]:
let inline (</>) a b = System.IO.Path.Combine (a, b)

## createTempDirectoryName

In [ ]:
let createTempDirectoryName () =
    let root =
        match System.Reflection.Assembly.GetEntryAssembly().GetName().Name with
        | assemblyName -> assemblyName

    System.IO.Path.GetTempPath ()
    </> $"!{root}"
    </> string (newGuidFromDateTime System.DateTime.Now)

In [ ]:
//// test

createTempDirectoryName ()
|> _contains System.IO.Path.DirectorySeparatorChar

C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-2901-0114-0d99add1e7f9


## createTempDirectory

In [ ]:
let createTempDirectory () =
    let tempFolder = createTempDirectoryName ()
    let result = System.IO.Directory.CreateDirectory tempFolder

    let getLocals () =
        $"tempFolder: {tempFolder} / result: {({|
            Exists = result.Exists
            CreationTime = result.CreationTime
        |})} {getLocals ()}"

    trace Debug (fun () -> "createTempDirectory") getLocals

    tempFolder

In [ ]:
//// test

let tempDirectory = createTempDirectory ()

Directory.Exists tempDirectory
|> _equal true

16:03:29 #1 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-2912-1244-1d013cf772c8 / result: { CreationTime = 2023-07-25 4:03:29 PM
  Exists = true }
True


## waitForFileAccess

In [ ]:
let waitForFileAccess path =
    let rec loop retry = async {
        try
            use _ = new System.IO.FileStream (
                path,
                System.IO.FileMode.Open,
                System.IO.FileAccess.ReadWrite,
                System.IO.FileShare.None
            )
            return retry
        with ex ->
            if retry % 100 = 0 then
                let getLocals () = $"path: {path} / ex: {ex |> printException} / {getLocals ()}"
                trace Warn (fun () -> "waitForFileAccess") getLocals
            do! Async.Sleep 10
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test.txt"

let lockFile () = async {
    trace Debug (fun () -> "_1") getLocals
    use stream = new System.IO.FileStream (path, System.IO.FileMode.Open, System.IO.FileAccess.ReadWrite)
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
    stream.Seek (0L, SeekOrigin.Begin) |> ignore
    trace Debug (fun () -> "_4") getLocals
    stream.WriteByte 49uy
    trace Debug (fun () -> "_5") getLocals
    stream.Flush ()
    trace Debug (fun () -> "_6") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    do! File.WriteAllTextAsync (path, "0") |> Async.AwaitTask
    trace Debug (fun () -> "2") getLocals
    let! child = lockFile () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    let! retries = waitForFileAccess path
    trace Debug (fun () -> "5") getLocals
    let! text = File.ReadAllTextAsync path |> Async.AwaitTask
    trace Debug (fun () -> "6") getLocals
    do! child
    trace Debug (fun () -> "7") getLocals
    return retries, text
}
|> Async.runWithTimeout 3000
|> function
    | Some (retries, text) ->
        retries
        |> _isBetween
            (if Runtime.isWindows () then 100 else 100)
            (if Runtime.isWindows () then 150 else 150)
        
        text |> _equal "1"
        
        true
    | _ -> false
|> _equal true

16:03:29 #2 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-2933-3364-3883491ac616 / result: { CreationTime = 2023-07-25 4:03:29 PM
  Exists = true }
16:03:29 #3 [Debug] 1
16:03:29 #4 [Debug] 2
16:03:29 #5 [Debug] _1
16:03:29 #6 [Debug] 3
16:03:29 #7 [Debug] _2
16:03:29 #8 [Debug] 4
16:03:29 #9 [Warn] waitForFileAccess / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-2933-3364-3883491ac616\test.txt / ex: System.IO.IOException: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-2933-3364-3883491ac616\test.txt' because it is being used by another process.
16:03:30 #10 [Warn] waitForFileAccess / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-2933-3364-3883491ac616\test.txt / ex: System.IO.IOException: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-2933-3364-3883491ac616\test.txt' because it is being u

## deleteDirectoryAsync

In [ ]:
let deleteDirectoryAsync path =
    let rec loop retry = async {
        try
            System.IO.Directory.Delete (path, true)
            return retry
        with ex ->
            if retry % 100 = 0 then
                let getLocals () = $"path: {path} / ex: {ex |> printException} / {getLocals ()}"
                trace Warn (fun () -> "deleteDirectoryAsync") getLocals
            do! Async.Sleep 10
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test"

let lockDirectory () = async {
    trace Debug (fun () -> "_1") getLocals
    System.IO.File.WriteAllText (path </> "test.txt", "0")
    use _ = new System.IO.FileStream (path </> "test.txt", System.IO.FileMode.Open, System.IO.FileAccess.ReadWrite)
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    Directory.CreateDirectory path |> ignore
    trace Debug (fun () -> "2") getLocals
    let! child = lockDirectory () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    let! retries = deleteDirectoryAsync path
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return retries
}
|> Async.runWithTimeout 3000
|> function
    | Some retries ->
        retries
        |> _isBetween
            (if Runtime.isWindows () then 100 else 100)
            (if Runtime.isWindows () then 150 else 150)

        true
    | _ -> false
|> _equal true

16:03:31 #18 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-3161-6101-6ebbfdeaaa89 / result: { CreationTime = 2023-07-25 4:03:31 PM
  Exists = true }
16:03:31 #19 [Debug] 1
16:03:31 #20 [Debug] 2
16:03:31 #21 [Debug] 3
16:03:31 #22 [Debug] _1
16:03:31 #23 [Debug] _2
16:03:31 #24 [Debug] 4
16:03:31 #25 [Warn] deleteDirectoryAsync / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-3161-6101-6ebbfdeaaa89\test / ex: System.IO.IOException: The process cannot access the file 'test.txt' because it is being used by another process.
16:03:33 #26 [Warn] deleteDirectoryAsync / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-3161-6101-6ebbfdeaaa89\test / ex: System.IO.IOException: The process cannot access the file 'test.txt' because it is being used by another process.
16:03:33 #27 [Debug] _3
16:03:33 #28 [Debug] 5
16:03:33 #29 [Debug] 6
128
128
128
True


## deleteFileAsync

In [ ]:
let deleteFileAsync path =
    let rec loop retry = async {
        try
            System.IO.File.Delete path
            return retry
        with ex ->
            if retry % 100 = 0 then
                let getLocals () = $"path: {path} / ex: {ex |> printException} / {getLocals ()}"
                trace Warn (fun () -> "deleteFileAsync") getLocals
            do! Async.Sleep 10
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test.txt"

let lockFile () = async {
    trace Debug (fun () -> "_1") getLocals
    use _ = new System.IO.FileStream (path, System.IO.FileMode.Open, System.IO.FileAccess.ReadWrite)
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    do! File.WriteAllTextAsync (path, "0") |> Async.AwaitTask
    trace Debug (fun () -> "2") getLocals
    let! child = lockFile () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    let! retries = deleteFileAsync path
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return retries
}
|> Async.runWithTimeout 3000
|> function
    | Some retries ->
        retries
        |> _isBetween
            (if Runtime.isWindows () then 100 else 100)
            (if Runtime.isWindows () then 150 else 150)

        true
    | _ -> false
|> _equal true

16:03:33 #30 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-3377-7772-74525a8bf63d / result: { CreationTime = 2023-07-25 4:03:33 PM
  Exists = true }
16:03:33 #31 [Debug] 1
16:03:33 #32 [Debug] 2
16:03:33 #33 [Debug] 3
16:03:33 #34 [Debug] _1
16:03:33 #35 [Debug] _2
16:03:33 #36 [Debug] 4
16:03:33 #37 [Warn] deleteFileAsync / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-3377-7772-74525a8bf63d\test.txt / ex: System.IO.IOException: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-3377-7772-74525a8bf63d\test.txt' because it is being used by another process.
16:03:35 #38 [Warn] deleteFileAsync / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-3377-7772-74525a8bf63d\test.txt / ex: System.IO.IOException: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-3377-7772-74525a8bf63d\test.txt' because it is bei

## moveFileAsync

In [ ]:
let moveFileAsync newPath oldPath =
    let rec loop retry = async {
        try
            System.IO.File.Move (oldPath, newPath)
            return retry
        with ex ->
            if retry % 100 = 0 then
                let getLocals () =
                    $"oldPath: {oldPath} / newPath: {newPath} / ex: {ex |> printException} / {getLocals ()}"
                trace Warn (fun () -> "moveFileAsync") getLocals
            do! Async.Sleep 10
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test.txt"
let newPath = tempFolder </> "test2.txt"

let lockFile () = async {
    trace Debug (fun () -> "_1") getLocals
    use _ = new System.IO.FileStream (path, System.IO.FileMode.Open, System.IO.FileAccess.ReadWrite)
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    do! File.WriteAllTextAsync (path, "0") |> Async.AwaitTask
    trace Debug (fun () -> "2") getLocals
    let! child = lockFile () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    let! retries = path |> moveFileAsync newPath
    trace Debug (fun () -> "5") getLocals
    let! text = File.ReadAllTextAsync newPath |> Async.AwaitTask
    trace Debug (fun () -> "6") getLocals
    do! child
    trace Debug (fun () -> "7") getLocals
    return retries, text
}
|> Async.runWithTimeout 3000
|> function
    | Some (retries, text) ->
        retries
        |> _isBetween
            (if Runtime.isWindows () then 100 else 100)
            (if Runtime.isWindows () then 150 else 150)
        
        text |> _equal "0"
        
        true
    | _ -> false
|> _equal true

16:03:35 #42 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-3596-9605-9a236e50a5b4 / result: { CreationTime = 2023-07-25 4:03:35 PM
  Exists = true }
16:03:35 #43 [Debug] 1
16:03:35 #44 [Debug] 2
16:03:35 #45 [Debug] 3
16:03:35 #46 [Debug] _1
16:03:35 #47 [Debug] _2
16:03:35 #48 [Debug] 4
16:03:35 #49 [Warn] moveFileAsync / oldPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-3596-9605-9a236e50a5b4\test.txt / newPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-3596-9605-9a236e50a5b4\test2.txt / ex: System.IO.IOException: The process cannot access the file because it is being used by another process.
16:03:37 #50 [Warn] moveFileAsync / oldPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-3596-9605-9a236e50a5b4\test.txt / newPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-3596-9605-9a236e50a5b4\test2.txt / ex: System.IO.IOException: The process cannot access the 

## watchDirectory

In [ ]:
[<RequireQualifiedAccess>]
type FileSystemChangeType =
    | Error
    | Changed
    | Created
    | Deleted
    | Renamed

[<RequireQualifiedAccess>]
type FileSystemChange =
    | Error of exn: exn
    | Changed of path: string * content: string option
    | Created of path: string * content: string option
    | Deleted of path: string
    | Renamed of oldPath: string * (string * string option)


let watchDirectoryWithFilter filter shouldReadContent path =
    let fullPath = System.IO.Path.GetFullPath path
    let getLocals () = $"fullPath: {fullPath} / filter: {filter} / {getLocals ()}"

    let watcher =
        new System.IO.FileSystemWatcher (
            Path = fullPath,
            NotifyFilter = filter,
            EnableRaisingEvents = true,
            IncludeSubdirectories = true
        )

    let getEventPath (path : string) =
        path |> String.trim |> String.replace fullPath "" |> String.trimStart [| '/'; '\\' |]

    let ticks () =
        System.DateTime.UtcNow.Ticks

    let changedStream =
        AsyncSeq.subscribeEvent
            watcher.Changed
            (fun event ->
                ticks (),
                [ FileSystemChange.Changed (getEventPath event.FullPath, None) ]
            )

    let deletedStream =
        AsyncSeq.subscribeEvent
            watcher.Deleted
            (fun event ->
                ticks (),
                [ FileSystemChange.Deleted (getEventPath event.FullPath) ]
            )

    let createdStream =
        AsyncSeq.subscribeEvent
            watcher.Created
            (fun event ->
                let path = getEventPath event.FullPath
                ticks (), [
                    FileSystemChange.Created (path, None)
                    if Runtime.isWindows () then
                        FileSystemChange.Changed (path, None)
                ])

    let renamedStream =
        AsyncSeq.subscribeEvent
            watcher.Renamed
            (fun event ->
                ticks (), [
                    FileSystemChange.Renamed (
                        getEventPath event.OldFullPath,
                        (getEventPath event.FullPath, None)
                    )
                ]
            )

    let errorStream =
        AsyncSeq.subscribeEvent
            watcher.Error
            (fun event -> ticks (), [ FileSystemChange.Error (event.GetException ()) ])


    let readContent fullPath = async {
        try
            do! waitForFileAccess fullPath |> Async.runWithTimeoutAsync 30000 |> Async.Ignore
            return! System.IO.File.ReadAllTextAsync fullPath |> Async.AwaitTask |> Async.map Some
        with ex ->
            trace Error (fun () -> $"watchWithFilter / readContent / ex: {ex |> printException}") getLocals
            return None
    }

    let stream =
        [
            changedStream
            deletedStream
            createdStream
            renamedStream
            errorStream
        ]
        |> FSharp.Control.AsyncSeq.mergeAll
        |> FSharp.Control.AsyncSeq.mapAsync (fun (n, events) -> async {
            let! events =
                if not shouldReadContent
                then events |> Async.init
                else 
                    events
                    |> List.map (fun event -> async {
                        match event with
                        | FileSystemChange.Changed (path, _) ->
                            let! content = fullPath </> path |> readContent
                            return FileSystemChange.Changed (path, content)
                        | FileSystemChange.Created (path, _) ->
                            do! Async.Sleep 1
                            let! content = fullPath </> path |> readContent
                            return FileSystemChange.Created (path, content)
                        | FileSystemChange.Renamed (oldPath, (newPath, _)) ->
                            let! content = fullPath </> newPath |> readContent
                            return FileSystemChange.Renamed (oldPath, (newPath, content))
                        | _ -> return event
                    })
                    |> Async.Parallel
                    |> Async.map Array.toList

            return
                events
                |> List.fold
                    (fun (i, events) event ->
                        i + 1L,
                        (n + i, event) :: events)
                    (0L, [])
                |> snd
                |> List.rev
        })
        |> FSharp.Control.AsyncSeq.concatSeq

    let disposable =
        newDisposable (fun () ->
            trace Debug (fun () -> "watchWithFilter / Disposing watch stream") getLocals
            watcher.EnableRaisingEvents <- false
            watcher.Dispose ()
        )

    stream, disposable

let watchDirectory path =
    watchDirectoryWithFilter
        (System.IO.NotifyFilters.Attributes
        ||| System.IO.NotifyFilters.CreationTime
        ||| System.IO.NotifyFilters.DirectoryName
        ||| System.IO.NotifyFilters.FileName
        //  ||| System.IO.NotifyFilters.LastAccess
        //  ||| System.IO.NotifyFilters.LastWrite
        ||| System.IO.NotifyFilters.Security
        //  ||| System.IO.NotifyFilters.Size
        )
        path

### testEventsRaw (test)

In [ ]:
//// test

let testEventsRaw
    (watchFn : bool -> string -> FSharp.Control.AsyncSeq<int64 * FileSystemChange> * IDisposable)
    write
    =
    let tempDirectory = createTempDirectory ()
    let stream, disposable = watchFn true tempDirectory

    let events = System.Collections.Concurrent.ConcurrentBag ()

    let iter () =
        stream
        |> FSharp.Control.AsyncSeq.iterAsyncParallel (fun event -> async { events.Add event })

    let run = async {
        let! _ = iter () |> Async.StartChild
        do! Async.Sleep 150
        return! write tempDirectory
    }

    try
        run
        |> Async.runWithTimeout 30000
        |> _equal (Some ())
    finally
        disposable.Dispose ()
        deleteDirectoryAsync tempDirectory |> Async.Ignore |> Async.RunSynchronously

    let eventsLog =
        events
        |> Seq.toList
        |> List.sortBy fst
        |> List.fold
            (fun (prev, acc) (ticks, event) ->
                ticks, (ticks, (if prev = 0L then 0L else ticks - prev), event) :: acc
            )
            (0L, [])
        |> snd
        |> List.rev
        |> List.map (fun (diff, n, event) ->
            let text = $"{n} / {diff} / {event}"
            if text |> String.length <= 100
            then text
            else text |> String.substring 0 100
        )
        |> String.concat "\n"
    let getLocals () = $"eventsLog: \n{eventsLog} / {getLocals ()}"
    trace Debug (fun () -> "testEventsRaw") getLocals

    events
    |> Seq.toList
    |> List.sortBy fst
    |> List.map snd
    |> List.fold
        (fun (map, acc) event ->
            match event with
            | FileSystemChange.Changed (path, (Some _ as content)) ->
                if map |> Map.containsKey path && map.[path] = content
                then map, acc
                else (map |> Map.add path content), event :: acc
            | FileSystemChange.Created (path, _)
            | FileSystemChange.Deleted path ->
                (map |> Map.remove path), event :: acc
            | FileSystemChange.Renamed (oldPath, (newPath, content)) ->
                (map |> Map.remove oldPath |> Map.remove newPath), event :: acc
            | _ -> map, event :: acc
        )
        (Map.empty, [])
    |> snd
    |> List.rev

#### fast (test)

In [ ]:
//// test

let write path = async {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"a{i}") |> Async.AwaitTask

    do! Async.Sleep 250

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"b{i}") |> Async.AwaitTask

    do! Async.Sleep 250

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt") |> Async.Ignore

    do! Async.Sleep 250

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file_{i}.txt", $"c{i}") |> Async.AwaitTask

    do! Async.Sleep 250

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt") |> Async.Ignore

    do! Async.Sleep 250
}

let run () =
    let events = testEventsRaw watchDirectory write

    events
    |> _sequenceEqual [
        FileSystemChange.Created ("file1.txt", Some "a1")
        FileSystemChange.Changed ("file1.txt", Some "a1")
        FileSystemChange.Created ("file2.txt", Some "a2")
        FileSystemChange.Changed ("file2.txt", Some "a2")

        FileSystemChange.Changed ("file1.txt", Some "b1")
        FileSystemChange.Changed ("file2.txt", Some "b2")

        FileSystemChange.Renamed ("file1.txt", ("file_1.txt", Some "b1"))
        FileSystemChange.Renamed ("file2.txt", ("file_2.txt", Some "b2"))

        FileSystemChange.Changed ("file_1.txt", Some "c1")
        FileSystemChange.Changed ("file_2.txt", Some "c2")

        FileSystemChange.Deleted "file_1.txt"
        FileSystemChange.Deleted "file_2.txt"
    ]

run
|> retryFn 3
|> _equal (Some ())

16:03:38 #55 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-3854-5485-56058fa1b75b / result: { CreationTime = 2023-07-25 4:03:38 PM
  Exists = true }
FSharpOption<Unit>
      Value: <null>
16:03:40 #56 [Debug] watchWithFilter / Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-3854-5485-56058fa1b75b / filter: FileName, DirectoryName, Attributes, CreationTime, Security
16:03:40 #57 [Debug] testEventsRaw / eventsLog: 
0 / 638259086187257100 / Created ("file1.txt", Some "a1")
1 / 638259086187257101 / Changed ("file1.txt", Some "a1")
38632 / 638259086187295733 / Created ("file2.txt", Some "a2")
1 / 638259086187295734 / Changed ("file2.txt", Some "a2")
2427598 / 638259086189723332 / Changed ("file1.txt", Some "b1")
9680 / 638259086189733012 / Changed ("file2.txt", Some "b2")
2483832 / 638259086192216844 / Renamed ("file1.txt", ("file_1.txt", Some "b1"))
3663 / 638259086192220507 / Renamed (

#### slow (test)

In [ ]:
//// test

let write path = async {
    let n = 2

    let contents =
        [1..n]
        |> List.map (string >> String.replicate 1_000_000)

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{contents.[i - 1]}a") |> Async.AwaitTask

    do! Async.Sleep 150

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{contents.[i - 1]}b") |> Async.AwaitTask

    do! Async.Sleep 150

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt") |> Async.Ignore

    do! Async.Sleep 150

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file_{i}.txt", $"{contents.[i - 1]}c") |> Async.AwaitTask

    do! Async.Sleep 150

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt") |> Async.Ignore

    do! Async.Sleep 150
}

let run () =
    let events =
        testEventsRaw watchDirectory write
        |> List.map (function
            | FileSystemChange.Changed (path, Some content) ->
                FileSystemChange.Changed (path, content |> Seq.distinct |> Seq.map string |> String.concat "" |> Some)
            | FileSystemChange.Created (path, Some content) ->
                FileSystemChange.Created (path, content |> Seq.distinct |> Seq.map string |> String.concat "" |> Some)
            | FileSystemChange.Renamed (oldPath, (newPath, Some content)) ->
                FileSystemChange.Renamed (
                    oldPath,
                    (newPath, content |> Seq.distinct |> Seq.map string |> String.concat "" |> Some)
                )
            | event -> event
        )

    events
    |> _sequenceEqual [
        FileSystemChange.Created ("file1.txt", Some "1a")
        FileSystemChange.Changed ("file1.txt", Some "1a")
        FileSystemChange.Created ("file2.txt", Some "2a")
        FileSystemChange.Changed ("file2.txt", Some "2a")

        FileSystemChange.Changed ("file1.txt", Some "1b")
        FileSystemChange.Changed ("file2.txt", Some "2b")

        FileSystemChange.Renamed ("file1.txt", ("file_1.txt", Some "1b"))
        FileSystemChange.Renamed ("file2.txt", ("file_2.txt", Some "2b"))

        FileSystemChange.Changed ("file_1.txt", Some "1c")
        FileSystemChange.Changed ("file_2.txt", Some "2c")

        FileSystemChange.Deleted "file_1.txt"
        FileSystemChange.Deleted "file_2.txt"
    ]

run
|> retryFn 3
|> _equal (Some ())

16:03:40 #58 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-4025-2563-289b33a32545 / result: { CreationTime = 2023-07-25 4:03:40 PM
  Exists = true }
16:03:40 #59 [Warn] waitForFileAccess / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-4025-2563-289b33a32545\file1.txt / ex: System.IO.IOException: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-4025-2563-289b33a32545\file1.txt' because it is being used by another process.
16:03:40 #60 [Warn] waitForFileAccess / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-4025-2563-289b33a32545\file1.txt / ex: System.IO.IOException: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-4025-2563-289b33a32545\file1.txt' because it is being used by another process.
16:03:40 #61 [Warn] waitForFileAccess / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-16

### testEventsSorted (test)

In [ ]:
//// test

let sortEvent event =
    match event with
    | FileSystemChange.Error _ -> 0
    | FileSystemChange.Created _ -> 1
    | FileSystemChange.Changed _ -> 2
    | FileSystemChange.Renamed (_oldPath, _) -> 3
    | FileSystemChange.Deleted _ -> 4

let formatEvents events =
    events
    |> Seq.toList
    |> List.sortBy (snd >> sortEvent)
    |> List.choose (fun (ticks, event) ->
        match event with
        | FileSystemChange.Error _ ->
            None
        | FileSystemChange.Changed (path, _) ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Changed)
        | FileSystemChange.Created (path, _) ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Created)
        | FileSystemChange.Deleted path ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Deleted)
        | FileSystemChange.Renamed (_oldPath, (path, _)) ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Renamed)
    )
    |> List.sortBy (fun (_, path, _) -> path)
    |> List.distinctBy (fun (_, path, event) -> path, event)

let testEventsSorted (watchFn : string -> FSharp.Control.AsyncSeq<int64 * FileSystemChange> * IDisposable) write =
    let path = createTempDirectory ()
    let stream, disposable = watchFn path

    let events = System.Collections.Concurrent.ConcurrentBag ()

    let iter () =
        stream
        |> FSharp.Control.AsyncSeq.iterAsyncParallel (fun event -> async { events.Add event })

    let run = async {
        let! _ = iter () |> Async.StartChild
        do! Async.Sleep 150
        return! write path
    }

    try
        run
        |> Async.runWithTimeout 3000
        |> _equal (Some ())
    finally
        disposable.Dispose ()
        deleteDirectoryAsync path |> Async.Ignore |> Async.RunSynchronously

    let events = formatEvents events

    let eventMap =
        events
        |> List.map (fun (ticks, path, event) -> path, (event, ticks))
        |> List.groupBy fst
        |> List.map (fun (path, events) ->
            let event, _ticks =
                events
                |> List.map snd
                |> List.sortByDescending snd
                |> List.head

            path, event
        )
        |> Map.ofList

    let eventList =
        events
        |> List.map (fun (_ticks, path, event) -> path, event)

    eventMap, eventList

#### create and delete (test)

In [ ]:
//// test

let write path = async {
    let n = 3

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file{i}.txt") |> Async.Ignore
    
    do! Async.Sleep 150
}

let eventMap, eventList = testEventsSorted (watchDirectory false) write

[
    "file1.txt", nameof FileSystemChangeType.Created
    "file1.txt", nameof FileSystemChangeType.Changed
    "file1.txt", nameof FileSystemChangeType.Deleted

    "file2.txt", nameof FileSystemChangeType.Created
    "file2.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Deleted

    "file3.txt", nameof FileSystemChangeType.Created
    "file3.txt", nameof FileSystemChangeType.Changed
    "file3.txt", nameof FileSystemChangeType.Deleted
]
|> _sequenceEqual eventList

[
    "file1.txt", nameof FileSystemChangeType.Deleted
    "file2.txt", nameof FileSystemChangeType.Deleted
    "file3.txt", nameof FileSystemChangeType.Deleted
]
|> Map.ofList
|> _sequenceEqual eventMap

16:03:41 #65 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-4188-8808-87e648f07dce / result: { CreationTime = 2023-07-25 4:03:41 PM
  Exists = true }
FSharpOption<Unit>
      Value: <null>
16:03:42 #66 [Debug] watchWithFilter / Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-4188-8808-87e648f07dce / filter: FileName, DirectoryName, Attributes, CreationTime, Security
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file1.txt, Deleted )
( file2.txt, Created )
( file2.txt, Changed )
( file2.txt, Deleted )
( file3.txt, Created )
( file3.txt, Changed )
( file3.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Deleted
      - Key: file2.txt
        Value: Deleted
      - Key: file3.txt
        Value: Deleted


#### change (test)

In [ ]:
//// test

let write path = async {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", "") |> Async.AwaitTask

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file{i}.txt") |> Async.Ignore
    
    do! Async.Sleep 150
}

let eventMap, eventList = testEventsSorted (watchDirectory false) write

[
    "file1.txt", nameof FileSystemChangeType.Created
    "file1.txt", nameof FileSystemChangeType.Changed
    "file1.txt", nameof FileSystemChangeType.Deleted

    "file2.txt", nameof FileSystemChangeType.Created
    "file2.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Deleted
]
|> _sequenceEqual eventList

[
    "file1.txt", nameof FileSystemChangeType.Deleted
    "file2.txt", nameof FileSystemChangeType.Deleted
]
|> Map.ofList
|> _sequenceEqual eventMap

16:03:42 #67 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-4235-3513-3b1cc9066282 / result: { CreationTime = 2023-07-25 4:03:42 PM
  Exists = true }
FSharpOption<Unit>
      Value: <null>
16:03:42 #68 [Debug] watchWithFilter / Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-4235-3513-3b1cc9066282 / filter: FileName, DirectoryName, Attributes, CreationTime, Security
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file1.txt, Deleted )
( file2.txt, Created )
( file2.txt, Changed )
( file2.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Deleted
      - Key: file2.txt
        Value: Deleted


#### rename (test)

In [ ]:
//// test

let write path = async {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt") |> Async.Ignore

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt") |> Async.Ignore
    
    do! Async.Sleep 150
}

let eventMap, eventList = testEventsSorted (watchDirectory false) write

[
    "file1.txt", nameof FileSystemChangeType.Created
    "file1.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Created
    "file2.txt", nameof FileSystemChangeType.Changed

    "file_1.txt", nameof FileSystemChangeType.Renamed
    "file_1.txt", nameof FileSystemChangeType.Deleted

    "file_2.txt", nameof FileSystemChangeType.Renamed
    "file_2.txt", nameof FileSystemChangeType.Deleted
]
|> _sequenceEqual eventList

[
    "file1.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Changed
    "file_1.txt", nameof FileSystemChangeType.Deleted
    "file_2.txt", nameof FileSystemChangeType.Deleted
]
|> Map.ofList
|> _sequenceEqual eventMap

16:03:42 #69 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-4277-7763-7bc96d83c27c / result: { CreationTime = 2023-07-25 4:03:42 PM
  Exists = true }
FSharpOption<Unit>
      Value: <null>
16:03:43 #70 [Debug] watchWithFilter / Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-4277-7763-7bc96d83c27c / filter: FileName, DirectoryName, Attributes, CreationTime, Security
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file2.txt, Created )
( file2.txt, Changed )
( file_1.txt, Renamed )
( file_1.txt, Deleted )
( file_2.txt, Renamed )
( file_2.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Changed
      - Key: file2.txt
        Value: Changed
      - Key: file_1.txt
        Value: Deleted
      - Key: file_2.txt
        Value: Deleted


#### full (test)

In [ ]:
//// test

let write path = async {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", "") |> Async.AwaitTask

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt") |> Async.Ignore

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file_{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt") |> Async.Ignore
    
    do! Async.Sleep 150
}

let eventMap, eventList = testEventsSorted (watchDirectory false) write

[
    "file1.txt", nameof FileSystemChangeType.Created
    "file1.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Created
    "file2.txt", nameof FileSystemChangeType.Changed

    "file_1.txt", nameof FileSystemChangeType.Changed
    "file_1.txt", nameof FileSystemChangeType.Renamed
    "file_1.txt", nameof FileSystemChangeType.Deleted

    "file_2.txt", nameof FileSystemChangeType.Changed
    "file_2.txt", nameof FileSystemChangeType.Renamed
    "file_2.txt", nameof FileSystemChangeType.Deleted
]
|> _sequenceEqual eventList

[
    "file1.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Changed
    "file_1.txt", nameof FileSystemChangeType.Deleted
    "file_2.txt", nameof FileSystemChangeType.Deleted
]
|> Map.ofList
|> _sequenceEqual eventMap

16:03:43 #71 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-4327-2792-22d528f01c88 / result: { CreationTime = 2023-07-25 4:03:43 PM
  Exists = true }
FSharpOption<Unit>
      Value: <null>
16:03:43 #72 [Debug] watchWithFilter / Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-1603-4327-2792-22d528f01c88 / filter: FileName, DirectoryName, Attributes, CreationTime, Security
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file2.txt, Created )
( file2.txt, Changed )
( file_1.txt, Changed )
( file_1.txt, Renamed )
( file_1.txt, Deleted )
( file_2.txt, Changed )
( file_2.txt, Renamed )
( file_2.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Changed
      - Key: file2.txt
        Value: Changed
      - Key: file_1.txt
        Value: Deleted
      - Key: file_2.txt
        Value: Deleted
